In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [ ]:
#Cargo todas las tablas

df1 = pd.read_csv("autor_definitivo.csv")
df2 = pd.read_csv("compartir_tiempo_definitivo.csv")
df3 = pd.read_csv("contenidos_definitivo.csv")
df4 = pd.read_csv("fuente_definitivo.csv")
df5 = pd.read_csv("metadatos_definitivo.csv")
df6 = pd.read_csv("tiempo_definitivo.csv")
df7 = pd.read_csv("tipo_definitivo.csv")
df8 = pd.read_csv("pais_idioma_definitivo.csv")
df9 = pd.read_csv("valores_definitivo.csv")

In [ ]:
df8

,Unnamed: 0,t_id,pais,idioma
0,0,8476,United States,English
1,1,10294,Bangladesh,English
2,2,3608,Tanzania,English
3,3,10142,Thailand,English
4,4,875,Kenya,English
...,...,...,...,...
6330,6330,4490,Bangladesh,English
6331,6331,8062,South Africa,English
6332,6332,8622,China,English
6333,6333,4021,Ukraine,English


In [ ]:
#Elimino las columnas que empiezan por 'Unnamed' para eliminar 'Unnamed: 0'

df1 = df1.loc[:, ~df1.columns.str.startswith("Unnamed")]
df2 = df2.loc[:, ~df2.columns.str.startswith("Unnamed")]
df3 = df3.loc[:, ~df3.columns.str.startswith("Unnamed")]
df4 = df4.loc[:, ~df4.columns.str.startswith("Unnamed")]
df5 = df5.loc[:, ~df5.columns.str.startswith("Unnamed")]
df6 = df6.loc[:, ~df6.columns.str.startswith("Unnamed")]
df7 = df7.loc[:, ~df7.columns.str.startswith("Unnamed")]
df8 = df8.loc[:, ~df8.columns.str.startswith("Unnamed")]
df9 = df9.loc[:, ~df9.columns.str.startswith("Unnamed")]

In [ ]:
df8 = df8.rename(columns={
    't_id': 'tid'})

In [ ]:
#Uno todas las tablas por la clave 'id' (que puede llamarse diferente en cada tabla) a la primera tabla

df_unido = df1.merge(df2, left_on="ID", right_on="id_compartir", how="outer") \
              .merge(df3, left_on="ID", right_on="id_contenido", how="outer") \
              .merge(df4, left_on="ID", right_on="ID", how="outer") \
              .merge(df5, left_on="ID", right_on="t_id", how="outer") \
              .merge(df6, left_on="ID", right_on="t_id", how="outer") \
              .merge(df7, left_on="ID", right_on="t_id", how="outer") \
              .merge(df8, left_on="ID", right_on="tid", how="outer") \
              .merge(df9, left_on="ID", right_on="ID", how="outer")

In [ ]:
df_unido.sample()

#Veo que tengo columnas con la misma información, en específico las de id

,ID,Autor,autor_codificado,id_compartir,compartir_tiempo,fecha_compartir,hora_compartir,id_contenido,titulo,texto,...,favorito,t_id_y,tiempo,t_id,tipo,tid,pais,idioma,Valores,Valores_codificados
3168,5273,Michael Sharma,24,5273,286,2022-09-30,14:00:00,5273,VP vetting? Trump meets with Sen. Joni Ernst,Donald Trump on Monday spent part of his July ...,...,7573.0,5273,417,5273.0,Carta al editor,5273,Myanmar,English,REAL,1


In [ ]:
df_unido.shape

#La tabla tiene 6335 filas y 27 columnas

(6335, 27)

In [ ]:
df_unido.columns

Index(['ID', 'Autor', 'autor_codificado', 'id_compartir', 'compartir_tiempo',
       'fecha_compartir', 'hora_compartir', 'id_contenido', 'titulo', 'texto',
       'tematica', 'Fuente', 'fuente_codificada', 't_id_x', 'visitas',
       'compartir', 'duracion', 'favorito', 't_id_y', 'tiempo', 't_id', 'tipo',
       'tid', 'pais', 'idioma', 'Valores', 'Valores_codificados'],
      dtype='object')

In [ ]:
df_unido_final = df_unido.drop(columns=["id_compartir", "id_contenido", "t_id_x", "t_id_y", "t_id", "tid"])

In [ ]:
df_unido_final.sample()

,ID,Autor,autor_codificado,compartir_tiempo,fecha_compartir,hora_compartir,titulo,texto,tematica,Fuente,...,visitas,compartir,duracion,favorito,tiempo,tipo,pais,idioma,Valores,Valores_codificados
5328,8879,Alexander Lee,0,796,2022-10-22,22:00:00,BREAKING: Trump Rushed Off Stage After Alleged...,"With Election Day only three days away, tensio...",Política,The Informer,...,696326.0,6689.0,167.0,9345.0,443,Columna,Algeria,English,FAKE,0


In [ ]:
#Tomo la fecha y la hora de referencia
inicio = pd.Timestamp('2022-09-01 07:00:00')

# Creao una columna con la suma del tiempo en horas
df_unido_final['fecha_hora_tiempo'] = inicio + pd.to_timedelta(df_unido_final['tiempo'], unit='h')

# Separo en dos columnas: una para fecha y otra para hora
df_unido_final['fecha_tiempo'] = df_unido_final['fecha_hora_tiempo'].dt.date
df_unido_final['hora_tiempo'] = df_unido_final['fecha_hora_tiempo'].dt.time

#Elimino la columna que he creado de más
df_unido_final.drop(columns=['fecha_hora_tiempo'], inplace=True)

In [ ]:
df_unido_final.sample()

,ID,Autor,autor_codificado,compartir_tiempo,fecha_compartir,hora_compartir,titulo,texto,tematica,Fuente,...,duracion,favorito,tiempo,tipo,pais,idioma,Valores,Valores_codificados,fecha_tiempo,hora_tiempo
3105,5175,Benjamin Singh,5,641,2022-10-11,23:00:00,"#MemeOfTheWeek: Trump Asked 'The Gays,' And Go...","#MemeOfTheWeek: Trump Asked 'The Gays,' And Go...",Política,Metro News,...,156.0,6093.0,335,Reportaje,Algeria,English,REAL,1,2022-09-15,06:00:00


In [ ]:
df_unido_final['visitas'] = df_unido_final['visitas'].astype('Int64')
df_unido_final['compartir'] = df_unido_final['compartir'].astype('Int64')
df_unido_final['duracion'] = df_unido_final['duracion'].astype('Int64')
df_unido_final['favorito'] = df_unido_final['favorito'].astype('Int64')

In [ ]:
df_unido_final = df_unido_final.rename(columns={
    'Valores_x': 'Valores',
    'Valores_codificados_x': 'Valores_codificados'
})

In [ ]:
df_unido_final.sample()

,ID,Autor,autor_codificado,compartir_tiempo,fecha_compartir,hora_compartir,titulo,texto,tematica,Fuente,...,duracion,favorito,tiempo,tipo,pais,idioma,Valores,Valores_codificados,fecha_tiempo,hora_tiempo
3032,5062,Benjamin Singh,5,932,2022-11-11,11:00:00,Clinton fact-checked on 'truthful' claim in em...,Hillary Clinton is getting hammered for saying...,Política,Morning Star,...,<NA>,6628,776,Entrevista a fondo,Turkey,English,REAL,1,2022-10-03,15:00:00


In [ ]:
df_unido_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6335 non-null   int64 
 1   Autor                6335 non-null   object
 2   autor_codificado     6335 non-null   int64 
 3   compartir_tiempo     6335 non-null   int64 
 4   fecha_compartir      6335 non-null   object
 5   hora_compartir       6335 non-null   object
 6   titulo               6335 non-null   object
 7   texto                6335 non-null   object
 8   tematica             6335 non-null   object
 9   Fuente               6335 non-null   object
 10  fuente_codificada    6335 non-null   int64 
 11  visitas              6326 non-null   Int64 
 12  compartir            6286 non-null   Int64 
 13  duracion             5642 non-null   Int64 
 14  favorito             5975 non-null   Int64 
 15  tiempo               6335 non-null   int64 
 16  tipo  

In [ ]:
df_unido_final.to_csv("df_unido.csv", index=False)